In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm


# Text Preprocessing and NLP
import nltk

# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize

# Part-of-speech tagging
from nltk import pos_tag

# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer

In [2]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/inflaton/code/CrediNews


# Data Preparation (Loading CSV)

Load the processed_data `csv` file into pandas DataFrames
- `processed_data.csv` is loaded into `data` DataFrame (stemming has been performed to reduce processing time.)

In [3]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)
datasets

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 54441 examples [00:01, 36525.13 examples/s]
Generating test split: 6050 examples [00:00, 39727.37 examples/s]
Generating rewritten_train split: 54441 examples [00:01, 49239.71 examples/s]
Generating rewritten_test split: 6050 examples [00:00, 52140.30 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
    rewritten_train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten_test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
})

### Convolutional Neural network + Custom-trained word2vec word embeddings + 5-Fold Cross Validation + L2 Regularization + GridSearchCV


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Embedding,
    Conv1D,
    GlobalMaxPooling1D,
    Dense,
    Dropout,
    Input,
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from gensim.models import Word2Vec
from tensorflow.keras.regularizers import l2

# Set seeds for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


def train_word2vec_and_create_embeddings(
    train_texts, word_index, max_words, embedding_dim=100
):
    """Train Word2Vec on training data only and create embedding matrix"""
    # Train Word2Vec on training data only
    train_sentences = [text.split() for text in train_texts]
    word2vec_model = Word2Vec(
        sentences=train_sentences,
        vector_size=embedding_dim,
        window=5,
        min_count=2,
        workers=4,
    )

    # Create embedding matrix with correct dimensions
    vocab_size = min(max_words, len(word_index) + 1)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in word_index.items():
        if i < vocab_size:  # Only include words within max_words limit
            if word in word2vec_model.wv:
                embedding_matrix[i] = word2vec_model.wv[word]
            else:
                embedding_matrix[i] = np.random.normal(size=(embedding_dim,))

    return embedding_matrix


def create_model(
    max_sequence_length,
    vocab_size,
    embedding_dim,
    embedding_matrix,
    filters,
    dropout_rate,
):
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=True,
    )(input_layer)

    x = Conv1D(
        filters=filters, kernel_size=5, activation="relu", kernel_regularizer=l2(0.01)
    )(embedding_layer)

    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation="relu", kernel_regularizer=l2(0.01))(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(1, activation="sigmoid", kernel_regularizer=l2(0.01))(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

2025-02-19 09:26:48.528121: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 09:26:48.683456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739928408.739939    4859 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739928408.756541    4859 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 09:26:48.908038: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
def train_model(
    train_data,
    val_data,
    epochs=10,
    batch_size=64,
    filters=64,
    dropout_rate=0.3,
    grid_search=False,
):
    # Set seeds for reproducibility
    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    # Constants
    max_words = 10000
    max_sequence_length = 300
    embedding_dim = 100

    print(f"\nTraining with paras: filters={filters}, dropout_rate={dropout_rate}")

    train_texts = train_data["processed_full_content"]
    val_texts = val_data["processed_full_content"]
    y_train = train_data["label"]
    y_val = val_data["label"]

    # Fit tokenizer on training data only
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    # Convert texts to sequences
    X_train = pad_sequences(
        tokenizer.texts_to_sequences(train_texts), maxlen=max_sequence_length
    )
    X_val = pad_sequences(
        tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length
    )

    # Get vocab size for this fold
    vocab_size = min(max_words, len(tokenizer.word_index) + 1)

    # Create embedding matrix using training data only
    embedding_matrix = train_word2vec_and_create_embeddings(
        train_texts, tokenizer.word_index, max_words, embedding_dim
    )

    # Create and train model
    model = create_model(
        max_sequence_length=max_sequence_length,
        vocab_size=vocab_size,
        embedding_dim=embedding_dim,
        embedding_matrix=embedding_matrix,
        filters=filters,
        dropout_rate=dropout_rate,
    )

    # Train model
    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )

    # Evaluate using F1-score
    y_pred = (model.predict(X_val) > 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    # Store results
    result = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

    print("\nResult:")
    for key, value in result.items():
        print(f"\t{key}: {value}")

    return result if grid_search else model

In [6]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
from tqdm import tqdm


def evaluate_model(model, train_data, val_data):
    print("Evaluating Model")

    max_words = 10000
    max_sequence_length = 300

    train_texts = train_data["processed_full_content"]
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    y_val = val_data["label"]
    val_texts = val_data["processed_full_content"]

    X_val = pad_sequences(
        tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length
    )
    y_pred = (model.predict(X_val) > 0.5).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")

In [7]:
def do_grid_search(data):
    # Define parameter grid
    param_grid = {"filters": [64, 128], "dropout_rate": [0.2, 0.3, 0.4, 0.5]}

    # Initialize variables to track results
    results = []
    best_score = 0
    best_params = None

    # Perform grid search with cross-validation
    for filters in param_grid["filters"]:
        for dropout_rate in param_grid["dropout_rate"]:
            print(f"\nTesting filters={filters}, dropout_rate={dropout_rate}")

            # Initialize cross-validation
            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            fold_scores = []

            # Perform k-fold cross-validation
            for fold, (train_idx, val_idx) in enumerate(
                kfold.split(data["processed_full_content"], data["label"]), 1
            ):
                print(f"\nFold {fold}")

                # Split data
                train_data = data.iloc[train_idx]
                val_data = data.iloc[val_idx]

                result = train_model(
                    train_data,
                    val_data,
                    filters=filters,
                    dropout_rate=dropout_rate,
                    grid_search=True,
                )

                fold_score = result["f1_score"]
                fold_scores.append(fold_score)

                print(f"Fold {fold} F1-score: {fold_score:.4f}")

            # Calculate average score for this parameter combination
            avg_score = np.mean(fold_scores)
            print(f"Average F1-score: {avg_score:.4f}")

            # Store results
            results.append(
                {
                    "filters": filters,
                    "dropout_rate": dropout_rate,
                    "avg_f1_score": avg_score,
                    "fold_scores": fold_scores,
                }
            )

            # Update best parameters if necessary
            if avg_score > best_score:
                best_score = avg_score
                best_params = {"filters": filters, "dropout_rate": dropout_rate}

    # Print final results
    print("\nGrid Search Results:")
    for result in results:
        print(
            f"Filters: {result['filters']}, Dropout: {result['dropout_rate']}, "
            f"F1-score: {result['avg_f1_score']:.4f}"
        )

    print("\nBest Parameters:")
    print(f"Filters: {best_params['filters']}")
    print(f"Dropout Rate: {best_params['dropout_rate']}")
    print(f"Best F1-Score: {best_score:.4f}")

    results_df = pd.DataFrame(results)
    print("\nResults Summary:")
    print(results_df.sort_values("avg_f1_score", ascending=False))
    
    return best_params

In [8]:
train_data = datasets["train"].to_pandas()
val_data = datasets["test"].to_pandas()
data = pd.concat([train_data, val_data], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60491 entries, 0 to 60490
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   60491 non-null  int64 
 1   full_content            60491 non-null  object
 2   processed_full_content  60491 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [9]:
%%time

best_params = do_grid_search(data)
best_params


Testing filters=64, dropout_rate=0.2

Fold 1

Training with paras: filters=64, dropout_rate=0.2


I0000 00:00:1739928434.762438    4859 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45689 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/10


I0000 00:00:1739928436.642354    5120 service.cc:148] XLA service 0x7ff3e0006050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739928436.642635    5120 service.cc:156]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-02-19 09:27:16.665289: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739928436.743756    5120 cuda_dnn.cc:529] Loaded cuDNN version 90300


 79/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6558 - loss: 2.6951

I0000 00:00:1739928437.461134    5120 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


757/757 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8551 - loss: 1.2798 - val_accuracy: 0.9626 - val_loss: 0.3324
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9617 - loss: 0.2936 - val_accuracy: 0.9678 - val_loss: 0.2035
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9681 - loss: 0.1939 - val_accuracy: 0.9687 - val_loss: 0.1713
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9722 - loss: 0.1645 - val_accuracy: 0.9732 - val_loss: 0.1558
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9756 - loss: 0.1497 - val_accuracy: 0.9735 - val_loss: 0.1485
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9799 - loss: 0.1384 - val_accuracy: 0.9746 - val_loss: 0.1416
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9826 - loss: 0.1296 - val_accuracy: 0.9731 - val_loss: 0.1440
Epoch 8/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9848 - loss: 0.1222 - val_accuracy: 0.9737 - val_

{'filters': 64, 'dropout_rate': 0.4}

In [10]:
%%time

model = train_model(train_data, val_data, filters=best_params["filters"], dropout_rate=best_params["dropout_rate"])
model


Training with paras: filters=64, dropout_rate=0.4
Epoch 1/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8641 - loss: 1.1742 - val_accuracy: 0.9579 - val_loss: 0.2889
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9603 - loss: 0.2584 - val_accuracy: 0.9669 - val_loss: 0.1875
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9677 - loss: 0.1826 - val_accuracy: 0.9701 - val_loss: 0.1657
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9711 - loss: 0.1631 - val_accuracy: 0.9706 - val_loss: 0.1589
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9752 - loss: 0.1523 - val_accuracy: 0.9734 - val_loss: 0.1534
Epoch 6/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9796 - loss: 0.1407 - val_accuracy: 0.9731 - val_loss: 0.1517
Epoch 7/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9804 - loss: 0.1340 - val_accuracy: 0.9736 - val_loss: 0.1467
Epoch 8/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step -

<Functional name=functional_40, built=True>

In [ ]:
model.save("results/CNN_model_original_CUDA.keras")

In [12]:
# load model
from tensorflow.keras.models import load_model

model2 = load_model("results/CNN_model_original.keras")
model2.summary()

Model: "functional_40"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_40 (InputLayer)     │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_40 (Embedding)        │ (None, 300, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_40 (Conv1D)              │ (None, 296, 64)        │        32,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_40         │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_80 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,108,869 (11.86 MB)

 Trainable params: 1,036,289 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,072,580 (7.91 MB)

In [13]:
%%time

evaluate_model(model, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9764
Precision: 0.9790
Recall: 0.9668
F1: 0.9728
CPU times: user 6.24 s, sys: 111 ms, total: 6.35 s
Wall time: 6.35 s


In [14]:
%%time

evaluate_model(model2, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9764
Precision: 0.9790
Recall: 0.9668
F1: 0.9728
CPU times: user 6.5 s, sys: 74.1 ms, total: 6.57 s
Wall time: 6.57 s


In [15]:
model3 = load_model("results/CNN_model.keras")
model3.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 300, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 296, 64)        │        32,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_5          │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,108,869 (11.86 MB)

 Trainable params: 1,036,289 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,072,580 (7.91 MB)

In [16]:
%%time

evaluate_model(model3, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.9762
Precision: 0.9750
Recall: 0.9705
F1: 0.9727
CPU times: user 6.75 s, sys: 181 ms, total: 6.93 s
Wall time: 6.94 s


In [17]:
val_data_rewritten = datasets["rewritten_test"].to_pandas()
train_data_rewritten = datasets["rewritten_train"].to_pandas()
data_rewritten = pd.concat([train_data, train_data_rewritten, val_data, val_data_rewritten], ignore_index=True)
data_rewritten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120982 entries, 0 to 120981
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   label                   120982 non-null  int64 
 1   full_content            120982 non-null  object
 2   processed_full_content  120982 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [18]:
%%time

evaluate_model(model, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.8433
Precision: 0.9110
Recall: 0.7114
F1: 0.7989
CPU times: user 6.24 s, sys: 80.2 ms, total: 6.32 s
Wall time: 6.3 s


In [19]:
%%time

evaluate_model(model2, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.8433
Precision: 0.9110
Recall: 0.7114
F1: 0.7989
CPU times: user 6.23 s, sys: 122 ms, total: 6.35 s
Wall time: 6.37 s


In [20]:
%%time

evaluate_model(model3, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.8453
Precision: 0.9147
Recall: 0.7129
F1: 0.8013
CPU times: user 6.42 s, sys: 91.8 ms, total: 6.52 s
Wall time: 6.54 s


In [21]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


Testing filters=64, dropout_rate=0.2

Fold 1

Training with paras: filters=64, dropout_rate=0.2
Epoch 1/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8628 - loss: 0.9199 - val_accuracy: 0.9343 - val_loss: 0.2787
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9268 - loss: 0.2752 - val_accuracy: 0.9383 - val_loss: 0.2408
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9343 - loss: 0.2426 - val_accuracy: 0.9393 - val_loss: 0.2282
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9401 - loss: 0.2246 - val_accuracy: 0.9419 - val_loss: 0.2209
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9454 - loss: 0.2141 - val_accuracy: 0.9429 - val_loss: 0.2161
Epoch 6/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9480 - loss: 0.2048 - val_accuracy: 0.9434 - val_loss: 0.2130
Epoch 7/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9518 - loss: 0.1960 - val_accuracy: 0.9438 - val_loss: 

{'filters': 64, 'dropout_rate': 0.2}

In [22]:
%%time

train_data_combined = pd.concat([train_data, train_data_rewritten], ignore_index=True)
val_data_combined = pd.concat([val_data, val_data_rewritten], ignore_index=True)
model_combined = train_model(train_data_combined, val_data_combined, 
                             filters=best_params_rewritten["filters"], 
                             dropout_rate=best_params_rewritten["dropout_rate"])
model_combined.save("results/CNN_model_combined_CUDA.keras")


Training with paras: filters=64, dropout_rate=0.2
Epoch 1/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8543 - loss: 1.0269 - val_accuracy: 0.9250 - val_loss: 0.3008
Epoch 2/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9255 - loss: 0.2819 - val_accuracy: 0.9343 - val_loss: 0.2404
Epoch 3/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9340 - loss: 0.2415 - val_accuracy: 0.9379 - val_loss: 0.2279
Epoch 4/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9405 - loss: 0.2235 - val_accuracy: 0.9373 - val_loss: 0.2224
Epoch 5/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9445 - loss: 0.2119 - val_accuracy: 0.9391 - val_loss: 0.2149
Epoch 6/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9494 - loss: 0.1995 - val_accuracy: 0.9411 - val_loss: 0.2075
Epoch 7/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9512 - loss: 0.1912 - val_accuracy: 0.9398 - val_loss: 0.2090
Epoch 8/10
1702/1702 ━━━━━━━━━━━━━━━━━━

In [23]:
evaluate_model(model_combined, train_data_combined, val_data_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Accuracy: 0.9410
Precision: 0.9382
Recall: 0.9261
F1: 0.9321


In [24]:
evaluate_model(model_combined, train_data_combined, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9731
Precision: 0.9740
Recall: 0.9641
F1: 0.9691


In [25]:
evaluate_model(model_combined, train_data_combined, val_data_rewritten)

Evaluating Model


190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9089
Precision: 0.9021
Recall: 0.8882
F1: 0.8951


In [30]:
model_combined2 = load_model("results/CNN_model_combined_CUDA.keras")
model_combined2.summary()

Model: "functional_81"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_81 (InputLayer)     │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_81 (Embedding)        │ (None, 300, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_81 (Conv1D)              │ (None, 296, 64)        │        32,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_81         │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_162 (Dense)               │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_81 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_163 (Dense)               │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,108,869 (11.86 MB)

 Trainable params: 1,036,289 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,072,580 (7.91 MB)

In [31]:
evaluate_model(model_combined2, train_data_combined, val_data_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Accuracy: 0.9410
Precision: 0.9382
Recall: 0.9261
F1: 0.9321


In [32]:
evaluate_model(model_combined2, train_data_combined, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9731
Precision: 0.9740
Recall: 0.9641
F1: 0.9691


In [33]:
evaluate_model(model_combined2, train_data_combined, val_data_rewritten)

Evaluating Model


190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9089
Precision: 0.9021
Recall: 0.8882
F1: 0.8951
